<a href="https://colab.research.google.com/github/shyamsankar11102000/Automated_Stock_Trading_Ensemble13/blob/main/automatedStockTradingEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-m_iwng18
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-m_iwng18
     |████████████████████████████████| 153kB 8.9MB/s 
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-dsu3_hhk/pyfolio
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-dsu3_hhk/pyfolio
     |████████████████████████████████| 5.5MB 14.2MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
  Created wheel for finrl: filename=finrl-0.0.3-cp36-none-any.whl size=24417 sha256=afc0a6ca9d32f3fbf0b8c2417c15b4c89eb2b6f38b90f85ff907008ca2c7a628
  Stored in directory: /tmp/pip-ephem-wheel-cache-zl2vc2x5/wheels/9c/19/bf/c644def96612df1ad42c94d5304966797eaa3221dffc5efe0b
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=4a8831803c2fcfd968b70af7e1c0d1187f0e423a1de69eb4defad580eacfa841
 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.6/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
df_stocks=YahooDownloader(start_date='2009-01-01', end_date='2021-01-01', ticker_list=config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df_stocks.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.795913,746015200.0,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700.0,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.680935,7010200.0,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.342327,7117200.0,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600.0,CSCO,4


In [ ]:
x = df_stocks.tic.unique()
templ = []

for name, group in df_stocks.groupby('date'):
    g = group.tic.unique()
    if len(templ) == 0:
        templ = [i for i in g if i in x]
    else:
        templ = [i for i in g if i in templ]


data_merge = pd.DataFrame(columns=list(df_stocks.columns))
x = np.array(templ).reshape(-1,1)
temp_df = pd.DataFrame.from_records(x, columns=['tic'])

for name, group in df_stocks.groupby('date'):
    temp_df['date'] = name

    result_outer = pd.merge(group, temp_df,  on=['date', 'tic'])
    result_outer = result_outer.sort_values(['date', 'tic'], ignore_index=True)

    assert len(result_outer) == len(temp_df.tic.unique())
    data_merge = data_merge.append(result_outer)

df_stocks = data_merge

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df_stocks)

Successfully added technical indicators
Successfully added turbulence index


In [ ]:
processed.sort_values(['date','tic'],ignore_index=True).head(10)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.795913,746015200.0,AAPL,4,0.000000,3.021788,2.688037,100.000000,66.666667,100.000000,2.795913,2.795913,0.0
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700.0,AXP,4,0.002647,3.021788,2.688037,100.000000,66.666667,100.000000,2.854912,2.854912,0.0
2,2009-01-02,42.799999,45.560001,42.779999,33.680935,7010200.0,BA,4,0.001883,2.977231,2.739885,70.355297,46.762875,100.000000,2.858558,2.858558,0.0
3,2009-01-02,44.910000,46.980000,44.709999,32.342327,7117200.0,CAT,4,-0.000747,2.956137,2.733662,50.429222,-29.787322,43.608349,2.844899,2.844899,0.0
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600.0,CSCO,4,-0.000088,2.943962,2.750274,60.227071,-9.011779,48.358256,2.847118,2.847118,0.0
5,2009-01-02,74.230003,77.300003,73.580002,48.043262,13695900.0,CVX,4,-0.002483,2.935827,2.739595,47.932746,-27.258297,49.952758,2.837711,2.837711,0.0
6,2009-01-02,21.605234,22.060680,20.993229,14.527276,13251000.0,DD,4,-0.006656,2.942815,2.702267,40.237287,-101.625172,9.320986,2.822541,2.822541,0.0
7,2009-01-02,22.760000,24.030001,22.500000,20.597496,9796600.0,DIS,4,-0.010740,2.947626,2.667385,37.181797,-113.588052,3.614128,2.807505,2.807505,0.0
8,2009-01-02,84.019997,87.620003,82.190002,72.844467,14088500.0,GS,4,-0.017042,2.964730,2.610597,31.067778,-136.015440,17.745809,2.787663,2.787663,0.0
9,2009-01-02,23.070000,24.190001,22.959999,17.909452,14902500.0,HD,4,-0.024359,2.982624,2.548940,27.267367,-166.469995,43.234949,2.765782,2.765782,0.0


In [ ]:
train = data_split(processed, '2009-01-01','2019-01-01')
trade = data_split(processed, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 301


In [ ]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [ ]:
agent = DRLAgent(env = env_train)

**A2C**

In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=100000)

**DDPG**

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

**PPO**

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)

**TD3**

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

**SAC**


In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)


In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=80000)


**SUPERVISED LEARNING MODELS**